## SentenceBERT + 머신러닝

### SentenceBERT 모델을 이용하여 임베딩 행렬 생성

In [1]:
### 필요한 라이브러리 설치 및 import
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics.pairwise import linear_kernel
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMClassifier
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import tensorflow as tf
from transformers import pipeline, set_seed, BertTokenizer, TFBertForSequenceClassification
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import re
import nltk 
from nltk.corpus import stopwords
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
from sentence_transformers import SentenceTransformer

In [2]:
### 학습용 데이터 --> DataFrame 생성, 누락 데이터 제거

# Data Load

file_path = ('D:\Code\DataSets/ratings_train.txt')
df_train = pd.read_csv(file_path,sep='\t')

print(f'학습용 데이터 확인 : \n{df_train}')

print('*'*80)

file_path = 'D:\Code\DataSets/ratings_test.txt' 
df_test = pd.read_csv(file_path, sep='\t')

print(f'평가용 데이터 확인 : \n{df_test}')

# 각 컬럼별 누락 데이터의 수 확인
num_nulls = df_train.isnull().sum()

df_train.dropna(inplace=True)

num_nulls2 = df_test.isnull().sum()

df_test.dropna(inplace=True)

df_train.reset_index(drop=True, inplace=True)

df_test.reset_index(drop=True, inplace=True)

print('*'*80)


print(df_train)

print('*'*80)

print(df_test)

학습용 데이터 확인 : 
              id                                           document  label
0        9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1        3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2       10265843                                  너무재밓었다그래서보는것을추천한다      0
3        9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4        6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
...          ...                                                ...    ...
149995   6222902                                인간이 문제지.. 소는 뭔죄인가..      0
149996   8549745                                      평점이 너무 낮아서...      1
149997   9311800                    이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?      0
149998   2376369                        청춘 영화의 최고봉.방황과 우울했던 날들의 자화상      1
149999   9619869                           한국 영화 최초로 수간하는 내용이 담긴 영화      0

[150000 rows x 3 columns]
***********************************************************

In [7]:
# 정규표현식 re를 쓰지않고 re.sub을 쓰지 않고 다르게 해보자!

# pattern 생성 : 한글 글자. 자음, 모음, 공백을 제외한 나머지 제거

import re
pattern = '[^가-힣ㄱ-ㅎㅏ-ㅣ ]'

# Series.str.replace(pattern, repl)
#df_train.loc[:, 'document'] = df_train.loc[:,'document'].str.replace(pattern, ' ')
df_train.loc[:, 'document'] = df_train.loc[:,'document'].apply(lambda x : re.sub(pattern, ' ', x))

print(df_train)

              id                                           document  label
0        9976970                                아 더빙   진짜 짜증나네요 목소리      0
1        3819312                  흠   포스터보고 초딩영화줄    오버연기조차 가볍지 않구나      1
2       10265843                                  너무재밓었다그래서보는것을추천한다      0
3        9045019                      교도소 이야기구먼   솔직히 재미는 없다  평점 조정      0
4        6483659  사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
...          ...                                                ...    ...
149990   6222902                                인간이 문제지   소는 뭔죄인가        0
149991   8549745                                      평점이 너무 낮아서         1
149992   9311800                    이게 뭐요  한국인은 거들먹거리고 필리핀 혼혈은 착하다       0
149993   2376369                        청춘 영화의 최고봉 방황과 우울했던 날들의 자화상      1
149994   9619869                           한국 영화 최초로 수간하는 내용이 담긴 영화      0

[149995 rows x 3 columns]


In [9]:

pattern = '[^가-힣ㄱ-ㅎㅏ-ㅣ ]'

#df_test.loc[:, 'document'] = df_train.loc[:,'document'].apply(lambda x : re.sub(pattern, ' ' , x))
df_test.loc[:, 'document'] = df_test.loc[:,'document'].str.replace(pattern, ' ')

print(df_test)

            id                                           document  label
0      6270596                                아 더빙   진짜 짜증나네요 목소리      1
1      9274899                  흠   포스터보고 초딩영화줄    오버연기조차 가볍지 않구나      0
2      8544678                                  너무재밓었다그래서보는것을추천한다      0
3      6825595                      교도소 이야기구먼   솔직히 재미는 없다  평점 조정      0
4      6723715  사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...      0
...        ...                                                ...    ...
49992  4608761                    보고 있는 것 자체가 괴로웠다  평가자체가 무의미한 영화      1
49993  5308387    공포 영화인데   효민 몸매만 기억에 남는다  도대체 이 연출력을 어찌할거냐    쩝      0
49994  9072549   이게 소위 스페인식             이란 건가    물론    아동 판타지 ...      0
49995  5802125                                   보지마셈 저 믿으세요 보지마셈      0
49996  6070594                                               최고졸작      0

[49997 rows x 3 columns]


In [11]:
### 사전 학습된 한국어 SentenceBERT 모델 생성(다운로드)
model_name='ddobokki/klue-roberta-base-nli-sts'
model = SentenceTransformer(model_name)

In [17]:
### 학습용 데이터(document) --> 문장 임베딩 행렬 생성 및 결과 저장하기

#
#def sentence_enc(sentence):
#    return model.encode(sentence)

# 너무 간단하니 lamba:x로 가능하지않을까? 사용자정의 함수를 굳이 하지 않는다?
#
#df_train.loc[:, 'document'].apply(lambda x:model.encode(x))

# sentences 생성
sentences = df_train.loc[:,'document'].values
print(sentences)

# model.encode(sentences)
train_embeddings = model.encode(sentences)

print('*'*80)

# 결과 확인하기
print(f'학습용 전체 리뷰에 대한 임베딩 행렬의 모양 : {train_embeddings.shape}')

['아 더빙   진짜 짜증나네요 목소리' '흠   포스터보고 초딩영화줄    오버연기조차 가볍지 않구나'
 '너무재밓었다그래서보는것을추천한다' ... '이게 뭐요  한국인은 거들먹거리고 필리핀 혼혈은 착하다 '
 '청춘 영화의 최고봉 방황과 우울했던 날들의 자화상' '한국 영화 최초로 수간하는 내용이 담긴 영화']


In [ ]:
### 학습용 데이터에 대한 임베딩 결과 저장하기

train_embeddings_path = 'D:\Code\Deep Learning/train_embeddings.npy ' 

# 결과 저장하기
np.save(train_embeddings_path, train_embeddings) #앞에는 경로 뒤에는 데이터